In [1]:
import re
import pandas as pd
import numpy as np
import joblib

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import gensim
import nltk

from nltk.stem.snowball import SnowballStemmer

from gensim.corpora import Dictionary, MmCorpus
from gensim.models import ldamodel

from sklearn.cluster import KMeans
from sklearn.manifold import MDS
from sklearn.decomposition import PCA

PATH = "D:/Petakabar/what/"
bencana = "bencana/"
ekonomi = "ekonomi/"
kecelakaan = "kecelakaan/"
kesehatan = "kesehatan/"
kriminalitas = "kriminalitas/"
olahraga = "olahraga/"

In [2]:
old = pd.read_csv('perbandingan/berita-24-5.csv')
# print(old)
new = pd.read_csv('perbandingan/berita-24-5.csv') # need changes
# print(new)

In [3]:
# old.columns
old.columns

Index(['ID', 'berita_title', 'berita_date', 'berita_qdate', 'berita_desc',
       'berita_source', 'created_at', 'berita_topik_id', 'qe_what', 'ner_when',
       'ner_who', 'ner_prov', 'ner_kab', 'ner_kec', 'ner_neg', 'sev_death',
       'sev_injury', 'sev_lost', 'sev_sink', 'sev_plane', 'sev_moneyUSD',
       'sev_moneyIDR', 'sev_persen', 'sev_inflasi', 'sev_internasional',
       'sev_nasional', 'sev_provkota', 'sev_lost_items', 'sev_lost_money',
       'sev_rape', 'sev_house', 'sev_person', 'class_classification',
       'berita_summary'],
      dtype='object')

# Bencana

In [3]:
# Bencana Section (Topic id = 1)
old_bencana = old[old['berita_topik_id'] == 1]
old_bencana = old_bencana[['berita_title', 'berita_date', 'berita_desc', 'berita_source']]
old_bencana.rename(columns = {'berita_title':'title', 'berita_date':'date', 'berita_desc':'desc', 'berita_source':'source'}, inplace=True)
print(old_bencana.shape)
old_bencana.head(1)

(2773, 4)


,title,date,desc,source
0,Bagaimana Proses Terjadinya Gempa Bumi?,"Kamis, 15 Des 2022 17:02 WIB","\- Saat gempa bumi terjadi, tanah dan apa saja...",www.kompas.com


## Preprocess

In [4]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
# previous way

nltk.download('punkt')
nltk.download('stopwords')

excluded_words = ["tempat", "waktu", "hari"]
excluded_words = []
NLTK_StopWords = stopwords.words('indonesian')
NLTK_StopWords.extend(["detik", "detikjatim", "detikjateng", "detikjabar", "detiksulsel", "detiksumbar", "detikbali",
                      "detikpapua", "detiksulteng", "detikmaluku", "detjatim", "detikcom", "allahumma", "aamiin", "allah", "bismillah"])
NLTK_StopWords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo',
                       'kalo', 'amp', 'biar', 'bikin', 'bilang',
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih',
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya',
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't',
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])
txt_stopword = pd.read_csv(PATH + bencana + "stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
NLTK_StopWords.extend(txt_stopword["stopwords"][0].split(' '))
NLTK_StopWords = set(NLTK_StopWords)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Archy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Archy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def preprocessing(berita):
        s = str(berita)
        s = s.lower()
        s = s.replace('\n', ' ')
        s = s.replace('\r', ' ')
        s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
        tokens = [token for token in s.split(" ") if token != ""]
        T = [t for t in tokens if (
            (t in excluded_words) or (t not in NLTK_StopWords))]
        return T

In [7]:
old_bencana = old_bencana.dropna(subset=['desc'])
old_bencana.sort_values('desc', inplace=True)
old_bencana.drop_duplicates(subset='desc', keep=False, inplace=True)
df_train_unprocessed, df_test_unprocessed = train_test_split(old_bencana,test_size=0.1)

old_bencana.head()

,title,date,desc,source
9646,"Imbas Banjir di Semarang, KA Tujuan Jakarta An...","Sabtu, 31 Des 2022 23:39 WIB",\r\n \r\n\- Imbas banjir yang menggenangi h...,www.tribunnews.com
648,"5 Titik Banjir di Tangerang Selatan Pagi Ini, ...","Jumat, 02 Des 2022 09:04 WIB",\r\n\- Banjir masih menggenangi sejumlah dae...,www.kompas.com
47,"Si Jago Merah Menyala di Balikpapan, Dua Rumah...","Senin, 28 Nov 2022 13:41 WIB",Api mendadak muncul dari salah satu rumah war...,www.kompas.com
1784,Banjir-Tanah Longsor Berulang hingga Menggenan...,"Rabu, 07 Des 2022 11:50 WIB",Badan Penanggulangan Bencana Daerah (BPBD) Ku...,www.kompas.com
585,"Banjir Natuna, Pemprov Kepri Bantu Rp 300 Juta...","Sabtu, 17 Des 2022 09:15 WIB",Banjir bandang melanda Kabupatan Natuna dan P...,www.kompas.com


In [9]:
df_train, df_test = train_test_split(old_bencana,test_size=0.1)
len(df_train)

2495

In [19]:
df_train.to_csv(PATH + bencana + "corpus/df_train_unprerocessed.csv" , index = None, header=True)
df_test.to_csv(PATH + bencana + "corpus/df_test_unprerocessed.csv" , index = None, header=True)

In [20]:
desc_text_test_unprocessed = []

for x in range(0, df_test_unprocessed.shape[0]):
  desc_text_test_unprocessed.append(df_test_unprocessed.iloc[x, -2])

joblib.dump(desc_text_test_unprocessed, PATH + bencana + "corpus/model/desc_text_test_unprocessed.pkl")

['D:/Petakabar/what/bencana/corpus/model/desc_text_test_unprocessed.pkl']

In [21]:
desc_text_train_unpreprocessed = []
desc_text_test_unpreprocessed = []

for x in range(0, df_train.shape[0]):
  desc_text_train_unpreprocessed.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test_unpreprocessed.append(df_test.iloc[x, -2])

In [22]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test_unpreprocessed, PATH + bencana + "corpus/model/desc_text_test_unpreprocessed.pkl")
joblib.dump(desc_text_train_unpreprocessed, PATH + bencana + "corpus/model/desc_text_train_unpreprocessed.pkl")

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
old_bencana.to_csv(PATH + bencana + 'corpus/dataset/df_total_unpreprocessed.csv', index=False)
df_test.to_csv(PATH + bencana + 'corpus/dataset/df_test_unpreprocessed.csv', index=False)
df_train.to_csv(PATH + bencana + 'corpus/dataset/df_train_unpreprocessed.csv', index=False)

In [8]:
for x in range(0, old_bencana.shape[0]):
    text = preprocessing(old_bencana.iloc[x, -2])
    old_bencana.iloc[x, -2] = (' '.join(text)) 

In [9]:
old_bencana.head()

,title,date,desc,source
9646,"Imbas Banjir di Semarang, KA Tujuan Jakarta An...","Sabtu, 31 Des 2022 23:39 WIB",imbas banjir menggenangi wilayah kota semarang...,www.tribunnews.com
648,"5 Titik Banjir di Tangerang Selatan Pagi Ini, ...","Jumat, 02 Des 2022 09:04 WIB",banjir menggenangi daerah kota tangerang selat...,www.kompas.com
47,"Si Jago Merah Menyala di Balikpapan, Dua Rumah...","Senin, 28 Nov 2022 13:41 WIB",api mendadak muncul salah rumah warga 04 kelur...,www.kompas.com
1784,Banjir-Tanah Longsor Berulang hingga Menggenan...,"Rabu, 07 Des 2022 11:50 WIB",badan penanggulangan bencana daerah bpbd kulon...,www.kompas.com
585,"Banjir Natuna, Pemprov Kepri Bantu Rp 300 Juta...","Sabtu, 17 Des 2022 09:15 WIB",banjir bandang melanda kabupatan natuna pulau ...,www.kompas.com


In [25]:
desc_text_train = []
desc_text_test = []

for x in range(0, df_train.shape[0]):
  desc_text_train.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test.append(df_test.iloc[x, -2])

In [26]:
# print(df_test.info())
print("Panjang Dokumen Test : ", len(desc_text_test))

print ('-'*90)

# print(df_train.info())
print("Panjang Dokumen Train : ", len(desc_text_train))

Panjang Dokumen Test :  278
------------------------------------------------------------------------------------------
Panjang Dokumen Train :  2495


In [27]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test, PATH + bencana + "corpus/model/desc_text_test.pkl")
joblib.dump(desc_text_train, PATH + bencana + "corpus/model/desc_text_train_new.pkl") #INI

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
old_bencana.to_csv(PATH + bencana + 'corpus/dataset/df_total_new.csv', index=False) #INI
df_test.to_csv(PATH + bencana + 'corpus/dataset/df_test.csv', index=False)
df_train.to_csv(PATH + bencana + 'corpus/dataset/df_train.csv', index=False)

In [28]:
# Fit transform menggunakan TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_test_matrix = tfidf_vectorizer.fit_transform(desc_text_test)
tfidf_train_matrix = tfidf_vectorizer.fit_transform(desc_text_train)

print('tfidf_test_matrix')
print(tfidf_test_matrix.shape)
print('tfidf_train_matrix')
print(tfidf_train_matrix.shape)

tfidf_test_matrix
(278, 9398)
tfidf_train_matrix
(2495, 29277)


In [29]:
joblib.dump(tfidf_vectorizer, PATH + bencana + "corpus/vectorizer/vectorizer_new.pkl") #INI
joblib.dump(tfidf_test_matrix, PATH + bencana + "corpus/matrix/tfidf_test.pkl")
joblib.dump(tfidf_train_matrix, PATH + bencana + "corpus/matrix/tfidf_train_new.pkl") #INI

['D:/Petakabar/what/bencana/corpus/matrix/tfidf_train_new.pkl']

## Pencarian

In [93]:
df_total = pd.read_csv(PATH + bencana + 'corpus/dataset/df_total_new.csv')
df_total = df_total[pd.notnull(df_total['desc'])]
print(df_total.info())
print ('-'*90)

document_text_train= joblib.load(PATH + bencana + 'corpus/model/desc_text_train_new.pkl')
document_text_test= joblib.load(PATH + bencana + 'corpus/model/desc_text_test.pkl')
print(len(document_text_test))
print(len(document_text_train))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2773 entries, 0 to 2772
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   2773 non-null   object
 1   date    2773 non-null   object
 2   desc    2773 non-null   object
 3   source  2773 non-null   object
dtypes: object(4)
memory usage: 86.8+ KB
None
------------------------------------------------------------------------------------------
278
2495


In [94]:
kueri='gempa'
kueri=preprocessing(kueri)
kueri= [" ".join (kueri)]
print (kueri)

tfidf_matrix =joblib.load(PATH + bencana + "corpus/matrix/tfidf_train_new.pkl" )
tfidf_vectorizer = joblib.load(PATH + bencana + "corpus/vectorizer/vectorizer_new.pkl" ) 
query_vec= tfidf_vectorizer.transform(kueri)
results=cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
#print (results)
print("\n======================\n")
print("Top 10 most similar documents in corpus:")

j = 1

for i in results.argsort()[-10:][::-1]:
    print("No ID Dokumen  : ", i)
    print("Tanggal        : ", df_total.iloc[i,1])
    print("Isi berita     : ", df_total.iloc[i,2])
    print("(Score: %.4f) " % results[i])

['gempa']


Top 10 most similar documents in corpus:
No ID Dokumen  :  9
Tanggal        :  Minggu, 25 Des 2022 20:20 WIB
Isi berita     :  100 kapal pinisi kapal wisata jenis labuan bajo berlindung gugusan pulau menghindari gelombang melanda perairan labuan bajo pantauan lokasi minggu 25 12 2022 sore kapal kapal wisata menumpuk perairan sisi utara hotel meruorah labuan bajo kapal kapal wisata terlindung pulau depannya kepala kantor kesyahbandaran otoritas pelabuhan ksop kelas iii labuan bajo hasan sadili membenarkan kapal kapal wisata berlindung cuaca buruk melanda labuan bajo hasan sadili membenarkan kapal kapal wisata berlindung pulau cuaca buruk kapal kapal wisata diaz berlindung pulau sabtu 24 12 2022 hari cuaca buruk melanda labuan bajo hujan lebat disertai angin kencang gelombang perairan labuan bajo prakiraan badan meteorologi klimatologi geofisika bmkg besok cuaca buruk angkutan sungai danau penyeberangan asdp indonesia ferry cabang labuan bajo manggarai barat nusa tenggara tim

## YAKE TFIDF BERT

In [30]:
import gensim
# import pandas as pd
import csv
from yake import KeywordExtractor
import math
from operator import itemgetter
from nltk import tokenize
from nltk.tokenize import word_tokenize
kw_extractor = KeyBERT('distilbert-base-nli-mean-tokens')
embedder = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')
import scipy.sparse


In [31]:
df_total = pd.read_csv(PATH + bencana + 'corpus/dataset/df_total_new.csv')
df_total = df_total[pd.notnull(df_total['desc'])]
print(df_total.info())
print ('-'*90)
document_text= joblib.load(PATH + bencana + 'corpus/model/desc_text_train_new.pkl')
print(len(document_text))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2773 entries, 0 to 2772
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   2773 non-null   object
 1   date    2773 non-null   object
 2   desc    2773 non-null   object
 3   source  2773 non-null   object
dtypes: object(4)
memory usage: 86.8+ KB
None
------------------------------------------------------------------------------------------
2495


In [32]:
df_test = pd.read_csv(PATH + bencana + 'corpus/dataset/df_test.csv')
df_test = df_test[pd.notnull(df_test['desc'])]
print(df_test.info())
print ('-'*90)
document_text_test= joblib.load(PATH + bencana + 'corpus/model/desc_text_test.pkl')
print(len(document_text_test))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   278 non-null    object
 1   date    278 non-null    object
 2   desc    278 non-null    object
 3   source  278 non-null    object
dtypes: object(4)
memory usage: 8.8+ KB
None
------------------------------------------------------------------------------------------
278


In [33]:
df_train = pd.read_csv(PATH + bencana + 'corpus/dataset/df_train.csv')
df_train = df_train[pd.notnull(df_train['desc'])]
print(df_train.info())
print ('-'*90)
document_text_train= joblib.load(PATH + bencana + 'corpus/model/desc_text_train_new.pkl')
print(len(document_text_train))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2495 entries, 0 to 2494
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   2495 non-null   object
 1   date    2495 non-null   object
 2   desc    2495 non-null   object
 3   source  2495 non-null   object
dtypes: object(4)
memory usage: 78.1+ KB
None
------------------------------------------------------------------------------------------
2495


In [35]:
paths = PATH + bencana + 'idwiki_word2vec_300.model'

# maximal value dari cosine similarity dalah 1

def similar(kata, proseske):
    most_similars = id_w2v.wv.most_similar(kata)
    arr = []

    # proses masukin ke csv
    f = open(PATH + bencana + 'bow_what_hapus.csv', 'a', encoding='UTF8', newline='')

    # f1 = open(PATH + bencana + "less_or_equal_62.csv", "a", encoding='UTF8', newline='')
    # f2 = open(PATH + bencana + "more_62.csv", "a", encoding='UTF8', newline='')

    for x in range(len(most_similars)):
        words = most_similars[x][0]
        similarity = most_similars[x][1]
        # print(words)
        # print(similarity)
        result = [words, similarity]
        # if similarity > maxDistance:
        #     maxDistance = similarity
        # elif similarity < minDistance:
        #     minDistance = similarity

        # if round(similarity, 2) <= 0.62:
        #     f1writer = csv.writer(f1)
        #     f1writer.writerow(result)
        # else:
        #     f2writer = csv.writer(f2)
        #     f2writer.writerow(result)
        
        arr.append((words, similarity))

    writer = csv.writer(f)
    writer.writerow([str(proseske), kata, most_similars])

    f.close()
    # f1.close()
    # f2.close()

    return(arr)


def tree(arr, proseske):
    hasil = []
    for x in range(len(arr)):
        for y in range(10):
            words = arr[x][y][0]
            similarity = arr[x][y][1]
            # if round(similarity, 2) <= 0.62:
            tes = similar(words, proseske)
            hasil.append(tes)
    return(hasil)

id_w2v = gensim.models.word2vec.Word2Vec.load(paths)

f = open(PATH + bencana + 'bow_what_hapus.csv', 'w', encoding='UTF8')
writer = csv.writer(f)
writer.writerow(['tingkat setelah parent', 'parent', 'similarity'])

# f1 = open(PATH + bencana + "less_or_equal_62.csv", "w", encoding='UTF8', newline='')
# writer = csv.writer(f1)
# writer.writerow(['Word', 'Similarity'])
# f1.close()

# f2 = open(PATH + bencana + "more_62.csv", "w", encoding='UTF8', newline='')
# writer = csv.writer(f2)
# writer.writerow(['Word', 'Similarity'])
# f2.close()

f.close()
x = 0

tes = similar('bencana', x)
data = []
data.append(tes)

while(True):
    if(x <= 3):
        x = x+1
        coba = tree(data, x)
        data = coba
        continue
    else:
        # print('maxDistance ' + str(maxDistance) + '\n')
        # print('minDistance ' + str(minDistance) + '\n')
        break

## Main Course

In [56]:
# Load bow dataset

#jika dijalankan ulang, jangan lupa untuk men-deduplikasi, menghapus kosakata selain indonesia, dan Rencana Induk Perencanaan Bencana
df_bow_what = pd.read_csv(PATH + bencana + "bow/bow_what_old.csv")
df_bow_what.head()

,tingkat setelah parent,parent,similarity
0,2.0,banjir,"[('longsor', 0.7020419836044312), ('kekeringan..."
1,2.0,kelaparan,"[('wabah', 0.7254995703697205), ('epidemi', 0...."
2,2.0,gempa,"[('tsunami', 0.6986302137374878), ('letusan', ..."
3,2.0,kekeringan,"[('kelaparan', 0.6980215907096863), ('erosi', ..."
4,2.0,kebakaran,"[('ledakan', 0.6575784087181091), ('pemadaman'..."


In [57]:
# Ambil parent dari bow
bow_list_what = []

for i in range(0, df_bow_what.shape[0]):
  bow_list_what.append(df_bow_what.iloc[i, 1])

print("What ", bow_list_what[0])

print({
  'What': len(bow_list_what),
})

What  banjir
{'What': 268}


In [38]:
# Cari dok pertama Use data train
def cari_dokpertama(kueriAsli: str):
    kueriPre = preprocessing(kueriAsli)
    kueriPre = " ".join(kueriPre)
    hasilSearch = []
    
    tfidf_matrix = joblib.load(PATH + bencana + 'corpus/matrix/tfidf_train_new.pkl')
    tfidf_vectorizer = joblib.load(PATH + bencana + 'corpus/vectorizer/vectorizer_new.pkl')

    query_vec = tfidf_vectorizer.transform([kueriPre])
    results = cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
    for i in results.argsort()[-5:][::-1]:
        hasilSearch.append(df_total.iloc[i,-2])
    hasilSearch=". ".join(hasilSearch)
    
    return hasilSearch

In [39]:
##Keywords Extraction with YAKE
def keyword_yake(hasilSearch: str):
    keywordYake=[]

    k_extractor = KeywordExtractor(lan="id", n=1, top=50)
    k_extractor2 = KeywordExtractor(lan="id", n=2, top=50)
    keywords = k_extractor.extract_keywords(text=hasilSearch)
    # keywords = k_extractor2.extract_keywords(text=hasilSearch)
    keywords.extend(k_extractor2.extract_keywords(text=hasilSearch))
    keywordYake = [x for x, y in keywords]
    # keywordYake.append(keywords)
    print('*'*120)
    print('*'*120)
    print("Keyword yake")
    print(keywordYake)
    print('*'*120)
    print('*'*120)
    
    return keywordYake

In [40]:
#Keywords Extraction with TFIDF
def keyword_tfidf(hasilSearch: str):

    keywordtfidf=[]
    keywordtfidf2=[]

    total_words = re.sub(r'[^\w]', ' ', hasilSearch)
    total_words = total_words.lower().split()
    #print (total_words)
    total_word_length = len(total_words)
    total_sentences = tokenize.sent_tokenize(hasilSearch)
    total_sent_len = len(total_sentences)

    tf_score = {}
    for each_word in total_words:
        #print (each_word)
        each_word = each_word.replace('.','')
        if (each_word in excluded_words) or (each_word not in NLTK_StopWords):
            if each_word in tf_score:
                tf_score[each_word] += 1
            else:
                tf_score[each_word] = 1

    # Dividing by total_word_length for each dictionary element
    tf_score.update((x, y/int(total_word_length)) for x, y in tf_score.items())
    #print(tf_score)
    def check_sent(word, sentences): 
        final = [all([w in x for w in word]) for x in sentences] 
        sent_len = [sentences[i] for i in range(0, len(final)) if final[i]]
        return int(len(sent_len))

    idf_score = {}
    for each_word in total_words:
        #print (each_word)
        each_word = each_word.replace('.','')
        if (each_word in excluded_words) or (each_word not in NLTK_StopWords):
            if each_word in idf_score:
                idf_score[each_word] = check_sent(each_word, total_sentences)
            else:
                idf_score[each_word] = 1

    # Performing a log and divide
    idf_score.update((x, math.log(int(total_sent_len)/y)) for x, y in idf_score.items())

    #print(idf_score)
    tf_idf_score = {key: tf_score[key] * idf_score.get(key, 0) for key in tf_score.keys()}
    #print(tf_idf_score)
    def get_top_n(dict_elem, n):
        result = dict(sorted(dict_elem.items(), key = itemgetter(1), reverse = True)[:n]) 
        hasil =list(result.keys())
        #print(list(result.keys()))        
        return hasil
    #print(get_top_n(tf_idf_score, 25))
    #print(len(get_top_n(tf_idf_score, 1)))
    keywordtfidf.append(get_top_n(tf_idf_score, 25))
    for i in range(len(keywordtfidf)):
        #print (i)
        totalKw=0
        totalKw=len(keywordtfidf[i])
        for j in range(totalKw):
            #print (j)
            keywordtfidf2.append(keywordtfidf[i][j])
    
    print('*'*120)
    print('*'*120)
    print("Keyword TFIDF")
    print (keywordtfidf2)
    print('*'*120)
    print('*'*120)

    return keywordtfidf2

In [41]:
#Keywords Extraction with BERT
def keyword_bert(hasilSearch: str):

    keywordbert=[]

    #for j in range(len(array_text)):
    keyword1 = kw_extractor.extract_keywords(hasilSearch, top_n=50, keyphrase_ngram_range=(1, 1))
    keyword2 = kw_extractor.extract_keywords(hasilSearch, top_n=50, keyphrase_ngram_range=(1, 2))

    #print("Keywords of article\n", keywords)
    for i in range (0,len (keyword1)):
        keywordbert.append(keyword1[i][0])
        keywordbert.append(keyword2[i][0])
    
    print('*'*120)
    print('*'*120)
    print("Keyword Bert")
    print (keywordbert)
    print('*'*120)
    print('*'*120)
    
    return keywordbert

In [42]:
# Borda rangking
def rangking (keywordGabung, kueriAsli: str):
    kandidatFinalCek=[]
    kandidatFinalFix=[]
    
    for i in keywordGabung:
        if (i not in kandidatFinalCek and i!=0):
            kandidatFinalCek.append(i)
    queries=[kueriAsli]
    query_embeddings = embedder.encode(queries)
    corpus_embeddings4 = embedder.encode(kandidatFinalCek)
    # Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
    closest_n = 80
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings4, 'cosine')[0]
        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
        for idx, distance in results[0:closest_n]:
            kandidatFinalFix.append(kandidatFinalCek[idx])

    print('*'*120)
    print('*'*120)
    print ('Kandidat Final Fix Rank: ', kandidatFinalFix)
    print('*'*120)
    print('*'*120)

    return kandidatFinalFix

In [43]:
# Keyword bow
def keywordCustomBow(bowList, initialQuery: str):
    cekDuplicate = []
    kandidatFix = []

    for i in bowList:
        if(i not in cekDuplicate and i!=0):
            cekDuplicate.append(i)

    queries=[initialQuery]
    query_embeddings = embedder.encode(queries)
    corpus_embeddings4 = embedder.encode(cekDuplicate)
    
    # Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
    closest_n = 500
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings4, 'cosine')[0]
        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
        for idx, distance in results[0:closest_n]:
            kandidatFix.append(cekDuplicate[idx])
    
    print('*'*120)
    print('*'*120)
    print("Keyword BoW")
    print ('Kandidat BoW: ', kandidatFix)
    print('*'*120)
    print('*'*120)

    return kandidatFix

In [58]:
# Creating query for what, when, where, who
what_initial_query = "bencana apa yang terjadi dalam berita"

what_query = preprocessing(what_initial_query)
what_query = " ".join(what_query)
print (what_query)

bencana berita


In [45]:
# Prepare w data
def prepareWData(initial_query: str, bow_list):
    hasilkandidat = []
    keywordGabung = []
    qeGabungan = []
    kueriFix = []

    hasilSearch     = cari_dokpertama(initial_query)
    # (ini yake + tfidf + bert) = qe statistik
    keywordYake     = keyword_yake(hasilSearch) # 20
    keywordtfidf2   = keyword_tfidf(hasilSearch) # 20
    keywordbert     = keyword_bert(hasilSearch) # 20
    # ini qe bow
    keywordBoW      = keywordCustomBow(bow_list, initial_query)

    for keyword1 in keywordYake:
        keywordGabung.append(keyword1)
    for keyword2 in keywordtfidf2:
        keywordGabung.append(keyword2)
    for keyword3 in keywordbert:
        keywordGabung.append(keyword3)  

    # hasilrank = qe statistik
    hasilrank = rangking(keywordGabung, initial_query)
    
    for word1 in hasilrank:
        kueriFix.append(word1)

    for word2 in keywordBoW:
        kueriFix.append(word2)

    for word3 in kueriFix:
        hasilkandidat.append(word3)

    kueriFix = [preprocessing(i) for i in kueriFix]
    
    qeGabunganDelimiter = []

    for word4 in kueriFix:
        for subWord in word4:
            qeGabungan.append(subWord)
            qeGabunganDelimiter.append(subWord)

    qeGabunganDelimiter = list(dict.fromkeys(qeGabunganDelimiter))
    qeGabungan = list(dict.fromkeys(qeGabungan))

    # (hasil ranking + bow) = kandidat final
    qeGabungan = [" ".join(qeGabungan)]

    print('*'*120)
    qeStatistik = hasilrank
    qeBoW = keywordBoW

    return [qeGabungan, qeStatistik, qeBoW, qeGabunganDelimiter]

In [59]:
whatResultList = prepareWData(what_query, bow_list_what)
print("What")
print()
qeGabunganWhat = whatResultList[0]
qeStatistikWhat = whatResultList[1]
qeBoWWhat = whatResultList[2]
qeGabunganDelimiterWhat = whatResultList[3]

************************************************************************************************************************
************************************************************************************************************************
Keyword yake
['sungai', 'mencapai', 'kebakaran', 'hutan', 'ciujung', 'air', 'lebak', 'febby', 'siaga', 'longsor', 'tma', 'hujan', 'warga', 'deras', 'debit', 'hari', 'rumah', 'rusak', 'edy', 'bencana', 'sentimeter', 'ciberang', 'sumut', 'tanah', 'duddy', 'pupr', 'jembatan', 'rahmayadi', 'berstatus', 'kepala', 'status', 'kecamatan', 'jalan', 'tercatat', 'cuaca', 'aman', 'relawan', 'daerah', 'upt', 'dinas', 'pembangunan', 'kehutanan', 'dampri', 'kabupaten', 'video', 'lokasi', 'bantaran', 'wilayah', 'berat', 'april', 'ciberang ciujung', 'tma mencapai', 'kebakaran hutan', 'sungai ciujung', 'ciujung lebak', 'sungai', 'sungai ciberang', 'lebak febby', 'berstatus siaga', 'tercatat tma', 'sentimeter aman', 'aman debit', 'debit air', 'febby cuaca', 'ciujung 

In [60]:
print(qeGabunganDelimiterWhat)
print(qeGabunganWhat)
print(qeBoWWhat)
print(qeStatistikWhat)

['bencana', 'kebencanaan', 'penanggulangan', 'febby', 'cuaca', 'hulu', 'ekstrem', 'rusak', 'meluap', 'kehutanan', 'kedungsumber', 'ciberang', 'dampri', 'menghancurkan', 'kesiapsiagaan', 'mengguyur', 'siaga', 'kebakaran', 'sindangkerta', 'pengecekan', 'kesatuan', 'terpantau', 'pembangunan', 'keterangan', 'gununghalu', 'langganan', 'pratama', 'menimbun', 'deras', 'ciujung', 'wilayah', 'mengakibatkan', 'bantaran', 'pemukiman', 'tma', 'pepohonan', 'jati', 'temayang', 'lingkungan', 'manusia', 'hutan', 'mengimbau', 'lebak', 'kabupaten', 'pengendalian', 'daerah', 'diguyur', 'banten', 'kecamatan', 'kemiringannya', 'hujan', 'hari', 'status', 'tanah', 'upt', 'menyebabkan', 'menyiagakan', 'wib', 'sambungnya', 'warga', 'debit', 'airnya', 'pencegahan', 'mengkhawatirkan', 'jalan', 'menetapkan', 'dinas', 'menargetkan', 'pengelolaan', 'air', 'tercatat', 'sekitaran', 'perkecambahan', 'letusan', 'benturan', 'gesekan', 'badai', 'gempabumi', 'kelaparan', 'rawan', 'penghujan', 'banjir', 'letusannya', 'pela

In [65]:
# df_test_unpreprocessed = pd.read_csv(PATH + bencana + "corpus/df_test_unprerocessed.csv")

# hasil_test_what = []

# for i in range(0, len(df_test_unpreprocessed)):
#     hasilWhat = []

#     for key in qeGabunganDelimiterWhat:
#         cariW = re.findall(key, df_test_unpreprocessed['desc'].iloc[i])

#         if cariW:
#             hasilWhat.append(key)
#     hasil_test_what.append(hasilWhat)

# hasil_test_what

df_test = pd.read_csv(PATH + bencana + "df_test_results.csv")

hasil_test_what = []

for i in range(0, len(df_test)):
    hasilWhat = []

    for key in qeGabunganDelimiterWhat:
        cariW = re.findall(key, df_test['desc'].iloc[i])

        if cariW:
            hasilWhat.append(key)
    hasil_test_what.append(hasilWhat)

hasil_test_what

[['bencana',
  'wilayah',
  'menyebabkan',
  'air',
  'banjir',
  'genangan',
  'menggenangi'],
 ['hulu', 'kebakaran', 'keterangan', 'warga', 'terbakar'],
 ['bencana',
  'cuaca',
  'ekstrem',
  'rusak',
  'daerah',
  'kecamatan',
  'hari',
  'warga',
  'tercatat',
  'rawan',
  'banjir',
  'diterjang',
  'longsor',
  'kencang',
  'angin'],
 ['kebakaran', 'hujan', 'warga', 'nyambar', 'petir', 'terbakar', 'angin'],
 ['bencana',
  'cuaca',
  'ekstrem',
  'mengguyur',
  'deras',
  'wilayah',
  'hujan',
  'hari',
  'tanah',
  'jalan',
  'banjir',
  'genangan',
  'kilat',
  'meteor',
  'petir',
  'longsor',
  'kencang',
  'angin'],
 ['bencana',
  'kebakaran',
  'keterangan',
  'manusia',
  'daerah',
  'hujan',
  'tanah',
  'jalan',
  'banjir',
  'erupsi',
  'gempa',
  'tertimpa',
  'bumi',
  'getaran',
  'terjal',
  'longsor',
  'kencang',
  'angin'],
 ['wilayah', 'air', 'banjir'],
 ['terpantau',
  'warga',
  'debit',
  'jalan',
  'air',
  'banjir',
  'kemacetan',
  'bahaya'],
 ['meluap',
  '

In [67]:
# df_test_unpreprocessed['qe_what_old']=hasil_test_what
# df_test_unpreprocessed

df_test['qe_what_new']=hasil_test_what

df_test['tp_old'] = 0
df_test['tn_old'] = 0
df_test['tp_new'] = 0
df_test['tn_new'] = 0

df_test


,title,date,desc,source,qe_what_old,qe_what_new,tp_old,tn_old,tp_new,tn_new
0,BPBD Waspadai Gelombang Pasang Air Laut di Pes...,"Rabu, 21 Des 2022 14:30 WIB",\-\r\n\r\nBPBD Lamongan mewaspadai pasang air ...,www.detik.com,"['bencana', 'banjir', 'desa', 'pesisir', 'pung...","[bencana, wilayah, menyebabkan, air, banjir, g...",0,0,0,0
1,"Bar Bekas Holywings di Tanjung Duren Terbakar,...","Senin, 05 Des 2022 20:19 WIB",\- Helen's Bar yang dahulu bernama Holywings T...,www.kompas.com,"['kebakaran', 'akibat', 'mengerahkan', 'warga'...","[hulu, kebakaran, keterangan, warga, terbakar]",0,0,0,0
2,"Angin Kencang Terjang Permukiman di Sidrap, 35...","Selasa, 03 Jan 2023 19:43 WIB","\-\r\n\r\n35 rumah di Kabupaten Sidrap, Sulawe...",www.detik.com,"['bencana', 'cuaca', 'banjir', 'kencang', 'kec...","[bencana, cuaca, ekstrem, rusak, daerah, kecam...",0,0,0,0
3,Kondisi Terkini 52 Rumah di Manggarai Hangus T...,"Minggu, 18 Des 2022 11:42 WIB",\-\r\n\r\nSebanyak 52 rumah dilanda kebakaran ...,www.detik.com,"['kebakaran', 'bantuan', 'mengungsi', 'jam', '...","[kebakaran, hujan, warga, nyambar, petir, terb...",0,0,0,0
4,"BMKG Rilis Peringatan Dini, Hujan Deras dan An...","Jumat, 09 Des 2022 13:44 WIB","\- Badan Meteorologi, Klimatologi dan Geofisik...",www.kompas.com,"['bencana', 'cuaca', 'banjir', 'meteorologi', ...","[bencana, cuaca, ekstrem, mengguyur, deras, wi...",0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
273,Geolog UPN soal Dentuman Misterius di Gunungki...,"Jumat, 23 Des 2022 15:30 WIB",\-\r\n\r\nSuara dentuman misterius terdengar d...,www.detik.com,"['wilayah', 'gempa', 'runtuh', 'terkikis', 'fe...","[wilayah, tanah, menyebabkan, air, gempa, bumi...",0,0,0,0
274,"Mangsa Ayam, Piton 2,5 Meter 'Ditaklukan' Petu...","Kamis, 22 Des 2022 22:31 WIB",\-\r\n\r\nKampung Babakanbaru RT 03/06 Desa Pa...,www.detik.com,"['kebakaran', 'wilayah', 'lokasi', 'warga', 'd...","[kebakaran, wilayah, pemukiman, warga, kelapar...",0,0,0,0
275,"Jokowi Resmikan Bendungan Ciawi dan Sukamahi, ...","Jumat, 23 Des 2022 10:00 WIB",Presiden Joko Widodo meresmikan Bendungan Ciaw...,www.kompas.com,"['banjir', 'terdampak', 'waduk']","[pembangunan, langganan, air, banjir, tanggul]",0,0,0,0
276,"Cerita Siswa Terdampak Rob di Bonang Demak, ke...","Selasa, 13 Des 2022 10:23 WIB",\-\r\n\r\nBanjir rob yang terjadi di Kecamatan...,www.detik.com,"['banjir', 'wilayah', 'kabupaten', 'hari', 'lo...","[wilayah, kabupaten, hari, tanah, sambungnya, ...",0,0,0,0


In [78]:
# df_test_unpreprocessed.to_csv(PATH + bencana + "df_test_results.csv" , index = None, header=True)
df_test = df_test.drop(['source'], axis=1)
df_test.to_csv(PATH + bencana + "df_test_results_appended.csv" , index = None, header=True)

## LDA

In [40]:
import gensim
from gensim import corpora
from collections import Counter
import pyLDAvis.gensim
import pickle 
import pyLDAvis
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

In [11]:
def remove_number(text):
    return  re.sub(r"\d+", "", text)

In [12]:
new_bencana = old_bencana[['title', 'desc']]
new_bencana['desc'] = new_bencana['desc'].apply(remove_number)
new_bencana['desc'] = new_bencana['desc'].apply(word_tokenize)
new_bencana['fdist'] = new_bencana['desc'].apply(FreqDist)

In [13]:
print(len(new_bencana))
new_bencana.head(2)

2773


,title,desc,fdist
9646,"Imbas Banjir di Semarang, KA Tujuan Jakarta An...","[imbas, banjir, menggenangi, wilayah, kota, se...","{'imbas': 1, 'banjir': 1, 'menggenangi': 1, 'w..."
648,"5 Titik Banjir di Tangerang Selatan Pagi Ini, ...","[banjir, menggenangi, daerah, kota, tangerang,...","{'banjir': 9, 'menggenangi': 2, 'daerah': 2, '..."


In [14]:
print('Frequency Tokens : \n') 
print(new_bencana['fdist'].head().apply(lambda x : x.most_common()))

Frequency Tokens : 

9646    [(stasiun, 8), (semarang, 6), (kereta, 6), (pe...
648     [(banjir, 9), (pondok, 9), (air, 8), (pagi, 7)...
47      [(kebakaran, 8), (lokasi, 6), (warga, 5), (rum...
1784    [(banjir, 13), (sungai, 7), (air, 7), (hujan, ...
585     [(rumah, 15), (kelurahan, 8), (banjir, 7), (an...
Name: fdist, dtype: object


In [15]:
dictionary = corpora.Dictionary(new_bencana['desc'])
print(dictionary)

corpus = [dictionary.doc2bow(berita) for berita in new_bencana['desc']]

Dictionary<27851 unique tokens: ['air', 'anggrek', 'antrean', 'api', 'arah']...>


In [16]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

total_topics = 3 # jumlah topik yang akan di extract
number_words = 30 # jumlah kata per topik

In [29]:
from gensim.test.utils import datapath
temp_file = datapath(PATH + bencana + "model/lda_model")
#if new
# lda_model = Lda(corpus, num_topics=total_topics, id2word=dictionary, passes=50, random_state=1)
# lda_model.save(temp_file)
# load
from gensim import models
lda_model = models.ldamodel.LdaModel.load(temp_file)

lda_model.show_topics(num_topics=total_topics, num_words=number_words)


[(0,
  '0.024*"banjir" + 0.022*"warga" + 0.020*"rumah" + 0.015*"desa" + 0.015*"korban" + 0.013*"kecamatan" + 0.010*"air" + 0.009*"jalan" + 0.007*"sungai" + 0.006*"kabupaten" + 0.006*"lokasi" + 0.006*"longsor" + 0.006*"cianjur" + 0.006*"terdampak" + 0.005*"hujan" + 0.005*"bantuan" + 0.005*"akibat" + 0.004*"semarang" + 0.004*"video" + 0.004*"gempa" + 0.004*"rusak" + 0.004*"kelurahan" + 0.004*"wilayah" + 0.004*"bencana" + 0.004*"bpbd" + 0.004*"anak" + 0.003*"kepala" + 0.003*"tim" + 0.003*"meter" + 0.003*"kota"'),
 (1,
  '0.004*"gunung" + 0.003*"kapal" + 0.003*"kota" + 0.003*"bandung" + 0.003*"api" + 0.003*"video" + 0.003*"nabi" + 0.003*"salah" + 0.003*"bencana" + 0.003*"rp" + 0.003*"indonesia" + 0.003*"masyarakat" + 0.003*"kereta" + 0.003*"memiliki" + 0.002*"stasiun" + 0.002*"bumi" + 0.002*"gambas" + 0.002*"detik" + 0.002*"jalur" + 0.002*"swt" + 0.002*"manusia" + 0.002*"lahar" + 0.002*"perjalanan" + 0.002*"alam" + 0.002*"fenomena" + 0.002*"jalan" + 0.002*"ular" + 0.002*"semeru" + 0.002*"a

In [30]:
# Word Count of Topic Keywords

from collections import Counter
topics = lda_model.show_topics(formatted=False, num_topics=total_topics, num_words=number_words)
# topic = lda_model.show_topics(num_topics=total_topics, num_words=number_words)
# print((topics[0]))
# print((topic[0]))
data_flat = [w for w_list in new_bencana['desc'] for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df_imp_wcount = pd.DataFrame(out, columns=['word', 'id', 'importance', 'word_count']) 
print(df_imp_wcount)

       word  id  importance  word_count
0    banjir   0    0.023772        6278
1     warga   0    0.022201        4843
2     rumah   0    0.020152        4336
3      desa   0    0.014642        3016
4    korban   0    0.014611        2776
..      ...  ..         ...         ...
85   daerah   2    0.003965        1432
86    video   2    0.003859        1920
87  kencang   2    0.003742        1004
88    warga   2    0.003665        4843
89     jawa   2    0.003660        1352

[90 rows x 4 columns]


In [32]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=new_bencana['desc']):
    # Init output
    sent_topics_df = pd.DataFrame()
    # print(len(ldamodel[corpus]))

    dom = []
    perc = []
    keywords = []
    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # print(len(row))

        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num, topn=number_words)
                topic_keywords = ", ".join([word for word, prop in wp])
                dom.append(int(topic_num))
                perc.append(round(prop_topic,4))
                keywords.append(topic_keywords)
                # sent_topics_df = pd.concat([sent_topics_df, pd.DataFrame([pd.Series([int(topic_num), round(prop_topic,4), topic_keywords])])], ignore_index=True)
                # sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    # print(len(sent_topics_df))
    sent_topics_df['Dominant_Topic'] = dom
    sent_topics_df['Perc_Contribution'] = perc
    sent_topics_df['Topic_Keywords'] = keywords
    # sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    # sent_topics_df['Texts']= texts
    # print(len(sent_topics_df))
    return(sent_topics_df)

In [33]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=new_bencana['desc'])
df_topic_sents_keywords['Texts'] = 0

In [34]:
df_dominant_topic = df_topic_sents_keywords.reset_index()
# texts = []
for i in range(len(new_bencana['desc'])):
    # print(new_bencana['desc'].iloc[i])
    # texts.append(new_bencana['desc'].iloc[i])
    df_dominant_topic['Texts'].iloc[i] = new_bencana['desc'].iloc[i]
# df_dominant_topic['Texts'] = texts
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# df_topic_sents_keywords['Texts'].isna().value_counts()
df_dominant_topic.head(5)
df_dominant_topic.tail(5)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
2768,2768,0,0.9907,"banjir, warga, rumah, desa, korban, kecamatan,...","[wilayah, pantai, utara, jawa, pantura, jawa, ..."
2769,2769,0,0.7524,"banjir, warga, rumah, desa, korban, kecamatan,...","[semarang, kaline, banjir, lirik, lagu, jawa, ..."
2770,2770,2,0.9963,"gempa, hujan, jakarta, wilayah, bmkg, kebakara...","[badan, meteorologi, klimatologi, geofisika, b..."
2771,2771,2,0.9929,"gempa, hujan, jakarta, wilayah, bmkg, kebakara...","[gempa, berkekuatan, magnitudo, kota, jayapura..."
2772,2772,0,0.9822,"banjir, warga, rumah, desa, korban, kecamatan,...","[mahasiswa, tewas, tersengat, listrik, melinta..."


In [35]:
df_dominant_topic.to_csv(PATH + bencana + 'df_dominant_topic.csv')

In [36]:
pyLDAvis.enable_notebook()

In [41]:
from datetime import datetime
now = datetime.now() # current date and time

year = now.strftime("%Y")
print("year:", year)

month = now.strftime("%m")
print("month:", month)

day = now.strftime("%d")
print("day:", day)

time = now.strftime("%H:%M:%S")
print("time:", time)

date_time = now.strftime("%m/%d/%Y,%H:%M:%S")
print("date and time:",date_time)
LDAvis_html = 'ldavis_prepared_'+ str(total_topics)+"_"+str(number_words)+"_"+str(date_time)
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
with open(LDAvis_html, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

year: 2023
month: 06
day: 22
time: 20:34:59
date and time: 06/22/2023,20:34:59


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:19                                                                                   │
│                                                                                                  │
│   16 date_time = now.strftime("%m/%d/%Y,%H:%M:%S")                                               │
│   17 print("date and time:",date_time)                                                           │
│   18 LDAvis_html = 'ldavis_prepared_'+ str(total_topics)+"_"+str(number_words)+"_"+str(date_t    │
│ ❱ 19 LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)                    │
│   20 with open(LDAvis_html, 'wb') as f:                                                          │
│   21 │   │   pickle.dump(LDAvis_prepared, f)                                                     │
│   22                                                                                             │
│                                                                                                  │
│ c:\Users\Archy\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\gensim.py:124  │
│ in prepare                                                                                       │
│                                                                                                  │
│   121 │   See `pyLDAvis.prepare` for **kwargs.                                                   │
│   122 │   """                                                                                    │
│   123 │   opts = fp.merge(_extract_data(topic_model, corpus, dictionary, doc_topic_dist), kwar   │
│ ❱ 124 │   return vis_prepare(**opts)                                                             │
│   125                                                                                            │
│                                                                                                  │
│ c:\Users\Archy\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\_prepare.py:44 │
│ 0 in prepare                                                                                     │
│                                                                                                  │
│   437 │   # For a detailed discussion, see: https://github.com/cpsievert/LDAvis/pull/41          │
│   438 │   term_frequency = np.sum(term_topic_freq, axis=0)                                       │
│   439 │                                                                                          │
│ ❱ 440 │   topic_info = _topic_info(topic_term_dists, topic_proportion,                           │
│   441 │   │   │   │   │   │   │    term_frequency, term_topic_freq, vocab, lambda_step, R,       │
│   442 │   │   │   │   │   │   │    n_jobs, start_index)                                          │
│   443 │   token_table = _token_table(topic_info, term_topic_freq, vocab, term_frequency, start   │
│                                                                                                  │
│ c:\Users\Archy\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\_prepare.py:24 │
│ 8 in _topic_info                                                                                 │
│                                                                                                  │
│   245 │   │   'Total': term_frequency,                                                           │
│   246 │   │   'Category': 'Default'})                                                            │
│   247 │   default_term_info = default_term_info.sort_values(                                     │
│ ❱ 248 │   │   by='saliency', ascending=False).head(R).drop('saliency', 1)                        │
│   249 │   # Rounding Freq and Total to integer values to match LDAvis code:                      │
│   250 │   default_term_info['Freq'] = np.floor(default_term

In [26]:
with open(LDAvis_html, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 with open(LDAvis_html, 'rb') as f:                                                           │
│   2 │   LDAvis_prepared = pickle.load(f)                                                         │
│   3                                                                                              │
│                                                                                                  │
│ c:\Users\Archy\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiv │
│ eshell.py:284 in _modified_open                                                                  │
│                                                                                                  │
│    281 │   │   │   "you can use builtins' open."                                                 │
│    282 │   │   )                                                                                 │
│    283 │                                                                                         │
│ ❱  284 │   return io_open(file, *args, **kwargs)                                                 │
│    285                                                                                           │
│    286 class InteractiveShell(SingletonConfigurable):                                            │
│    287 │   """An enhanced, interactive shell for Python."""                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
OSError: [Errno 22] Invalid argument: 'ldavis_prepared_3_30_06/22/2023,20:08:53'

In [ ]:
pyLDAvis.save_html(LDAvis_prepared, LDAvis_html +'.html')

In [ ]:
# LDAvis_prepared

In [107]:
# df_imp_wcount.sort_values(by=['word_count'], ascending=False)
df_dominant_topic['Keywords'].iloc[2771]
# lda_model.show_topic(1, topn=15)

'gempa, bmkg, hujan, bumi, cianjur, bencana, cuaca, angin, laut, jawa, magnitudo, daerah, pusat, desember, kencang, potensi, kerusakan, gunung, informasi, akibat, bantuan, kedalaman, gambas, stasiun, detik, berpotensi, badan, ekstrem, kepala, karangasem, lebat, rusak, bangunan, dirasakan, susulan, titik, mengalami, fenomena, kondisi, ringan, aktivitas, kereta, dampak, meteorologi, selasa, geofisika, indonesia, wita, sesar, terkait'

In [108]:
lda_what_bencana = []
for i in range(len(df_dominant_topic)):
    val = []
    for n, key in enumerate(df_dominant_topic['Keywords'].iloc[i].split(", ")):
        if key in df_dominant_topic['Text'].iloc[i]:
            val.append(key)
    lda_what_bencana.append(val)

print(lda_what_bencana)


[['jawa', 'stasiun', 'mengalami', 'kereta'], ['banjir', 'bpbd', 'terdampak', 'kelurahan', 'ketinggian', 'daerah', 'kepala', 'terendam', 'titik', 'genangan', 'kawasan', 'surut', 'mengungsi', 'jumat', 'rw', 'sentimeter'], ['kebakaran', 'api', 'anak', 'terbakar', 'pemadam', 'barang', 'akibat', 'diduga', 'peristiwa', 'senin', 'listrik', 'kepala', 'kondisi'], ['banjir', 'jalan', 'sungai', 'hujan', 'bpbd', 'longsor', 'akibat', 'ketinggian', 'daerah', 'kepala', 'pohon', 'terendam', 'genangan', 'bencana', 'jembatan', 'rusak', 'deras', 'pemerintah', 'mencapai'], ['banjir', 'sungai', 'hujan', 'bpbd', 'terdampak', 'kelurahan', 'akibat', 'daerah', 'kepala', 'terendam', 'titik', 'bencana', 'bantuan', 'jiwa', 'sabtu', 'mencapai', 'jumat', 'bandang'], ['banjir', 'jalan', 'sungai', 'hujan', 'daerah', 'kondisi', 'titik', 'genangan', 'jembatan', 'deras', 'surut', 'tanggul', 'kendaraan'], ['banjir', 'sungai', 'terdampak', 'ketinggian', 'rob', 'kepala', 'terendam', 'kondisi', 'titik', 'surut', 'jawa', 'me

In [109]:
# text  = preprocessing(old_bencana['desc'][old_bencana['desc'].str.find('hari') != -1].iloc[0])
# text
print(df_dominant_topic['Keywords'].iloc[0])
print(type(df_dominant_topic['Keywords'].iloc[0]))
print(df_dominant_topic['Text'].iloc[0])
print(type(df_dominant_topic['Text'].iloc[0]))
print(df_dominant_topic['Keywords'].iloc[i].split(", "))


gempa, bmkg, hujan, bumi, cianjur, bencana, cuaca, angin, laut, jawa, magnitudo, daerah, pusat, desember, kencang, potensi, kerusakan, gunung, informasi, akibat, bantuan, kedalaman, gambas, stasiun, detik, berpotensi, badan, ekstrem, kepala, karangasem, lebat, rusak, bangunan, dirasakan, susulan, titik, mengalami, fenomena, kondisi, ringan, aktivitas, kereta, dampak, meteorologi, selasa, geofisika, indonesia, wita, sesar, terkait
<class 'str'>
['imbas', 'banjir', 'menggenangi', 'semarang', 'jawa', 'stasiun', 'kereta', 'api', 'tawang', 'perjalanan', 'kereta', 'api', 'lintas', 'mengalami', 'gangguan', 'antrean', 'kereta', 'api', 'tujuan', 'jakarta', 'terhindarkan', 'pantauan', 'tribun', 'sembrani', 'tujuan', 'surabaya', 'pasar', 'turi', 'gambir', 'tertahan', 'stasiun', 'karangjati', 'menit', 'pengeras', 'suara', 'mengumumkan', 'kereta', 'tertahan', 'menunggu', 'giliran', 'stasiun', 'semarang', 'tawang', 'maaf', 'kereta', 'tertahan', 'stasiun', 'karangjati', 'menunggu', 'antrean', 'stasiu